In [1]:
import json
import pandas as pd

In [2]:
file = open("../data/EventsWithoutSeries.json")
file_content = file.read()
file.close()

In [3]:
query_dump = json.loads(file_content)
df = pd.DataFrame(query_dump)

In [4]:
with_dblp = df[df["dblpEventId"].notna()]
# every entry has two /

In [5]:
import requests
import bs4
from bs4 import BeautifulSoup

In [6]:
def get_parent_page(dbpl_id: str, dbpl_base_url: str = "https://dblp.org/db/") -> str:
  url = dbpl_base_url + dbpl_id
  db_req = requests.get(url=url)
  soup = BeautifulSoup(db_req.text, 'html.parser')
  breadcrumbs = soup.find('div', {'id': 'breadcrumbs'})
  last_itemprop = breadcrumbs.find_all('span', {'itemprop': 'itemListElement'})[-1]
  return last_itemprop.find('a')['href']

In [7]:
def get_headline_of_dbpl_series(dbpl_series: str) -> str:
  dbpl_parent_page = requests.get(dbpl_series)
  if not dbpl_parent_page.text:
    raise ValueError("Parent page could not be reached:" + dbpl_series)
  parent_soup = BeautifulSoup(dbpl_parent_page.text)
  headline = parent_soup.find('header', {'id': 'headline'})
  return headline.find('h1').text

In [8]:
from typing import Optional


def get_series_name(dbpl_event_id: str) -> Optional[str]:
  try:
    parts = dbpl_event_id.split('/')
    parent_id = '/'.join(parts[:2])
    return get_headline_of_dbpl_series("https://dblp.org/db/" + parent_id)
  except:
    return None

In [10]:
with_dblp['dblpEventId'].iloc[:30].map(get_series_name)

15     Tunisian-Algerian Joint Conference on Applied ...
16     Conference on Computational Humanities Researc...
30     International Conference of the Italian Associ...
37            The Practice of Enterprise Modeling (PoEM)
39     International Workshop on Information Technolo...
56          International Semantic Web Conference (ISWC)
57     IEEE International Conference on Formal Engine...
60          International Semantic Web Conference (ISWC)
61          International Semantic Web Conference (ISWC)
62          International Semantic Web Conference (ISWC)
73     International Conference on Information and Kn...
74     International Conference on Information and Kn...
98     International Workshop on Knowledge Discovery ...
99     Deutsche Jahrestagung für Künstliche Intellige...
100    European Conference on Principles of Data Mini...
115    Italian Conference on Theoretical Computer Sci...
116    Conference and Labs of the Evaluation Forum (C...
117              Very Large Dat

In [11]:
with_dblp['label'].iloc[:30]

15     Tunisian-Algerian Joint Conference on Applied ...
16     Computational Humanities Research Conference 2022
30     1st Workshop on Artificial Intelligence for Cu...
37                PoEM 2022 Workshops and Models at Work
39     2nd International Workshop on Information Tech...
56                            SemREC 2022 and SMART 2022
57     International Workshop on Formal Analysis and ...
60     17th International Workshop on Ontology Matchi...
61     Seventh International Workshop on the Visualiz...
62     ISWC 2022 Posters, Demos and Industry Tracks: ...
73     Workshop on Deep Learning for Search and Recom...
74                                   CIKM 2022 Workshops
98     20th anniversary Workshop on Knowledge Discove...
99        8th Workshop on Formal and Cognitive Reasoning
100    MACLEAN: MAChine Learning for EArth Observatio...
115    23rd Italian Conference on Theoretical Compute...
116    CLEF 2022 - Conference and Labs of the Evaluat...
117                            